In [ ]:
import pandas as pd
import datetime as dt

df = pd.read_csv("../../data/Avtice-losses.csv", header = 1, parse_dates = ["Zeitstempel"])
df["Zeitstempel"] = df["Zeitstempel"] - dt.timedelta(minutes=15)
df = df.groupby("Zeitstempel", as_index=False).kWh.mean()
df = df.resample("H", on = "Zeitstempel").sum().reset_index()
data_loss = df.set_index("Zeitstempel")
data_loss

In [ ]:
data_renewable_generation = pd.read_csv("../../data/Forecast-renewable-generation.csv", header = 0, parse_dates = ["datetime"])
data_renewable_generation = data_renewable_generation.groupby("datetime", as_index=False)[[col for col in data_renewable_generation.columns]].mean()
data_renewable_generation = data_renewable_generation.set_index("datetime")
data_renewable_generation

In [ ]:
data_NTC = pd.read_csv("../../data/NTC.csv", header = 0, parse_dates = ["datetime"])
data_NTC = data_NTC.groupby("datetime", as_index=False)[[col for col in data_NTC.columns]].mean()
data_NTC = data_NTC.set_index("datetime")
data_NTC

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier
cal_fourier_gen_ = CalendarFourier("Y", 6)

fourier_data_yearly = cal_fourier_gen_.in_sample(data_loss.index)
fourier_data_yearly

In [ ]:
from statsmodels.tsa.deterministic import TimeTrend
trend_gen = TimeTrend(False, 1)
trend = trend_gen.in_sample(data_loss.index)
trend

In [ ]:
#data_new = data_loss.join([data_NTC, data_renewable_generation, fourier_data_yearly, trend])
data_new = data_loss.join([data_renewable_generation, fourier_data_yearly])
data_new

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

cols = data_new.columns

X = data_new.to_numpy()
X = X[[not any(x) for x in np.isnan(X)]]
X = pd.DataFrame(X, columns = cols)
normalized_X=(X-X.mean())/X.std()
train_df, test_df = train_test_split(normalized_X.to_numpy(), test_size=0.2)

In [ ]:
import torch
from torch import optim, Tensor, nn
import lightning
from lightning.pytorch import LightningModule
import torch.nn.functional as F
from torchmetrics.regression import MeanSquaredError
import pickle

class SimpleModel(LightningModule):
    def __init__(self, **kwargs):
        super().__init__()

        default_dict = {"epochs":1, "batch_size":2, "lr":0.001, "momentum": None, "optimizer":optim.Adam, 
                        "loss":F.mse_loss, "metrics":[MeanSquaredError]}
        self.param_dict = {}
        #Setting the parameter either from kwrags or by default
        for key in default_dict.keys(): 
            if key in kwargs.keys():
                setattr(self, key, kwargs[key])
                self.param_dict[key] = kwargs[key]
            else:
                setattr(self, key, default_dict[key])
                self.param_dict[key] = default_dict[key]

        # Network Layer
        self.l1 = nn.LSTM(17,20,1)
        self.l2 = nn.LSTM(20, 8,1)
        self.l4 = nn.Linear(8*24*7, 128)
        self.l5 = nn.Linear(128,32)
        self.l6 = nn.Linear(32,24)

        self.activation_elu = nn.ELU()
        self.activation_relu = nn.ReLU()
        # logs 
        
        # Important: This property activates manual optimization.
        # self.automatic_optimization = False

    def forward(self, x):
        x = nn.ReLU()(self.l1(x)[0])
        x = nn.ReLU()(self.l2(x)[0])
        x = torch.flatten(x, start_dim=1)
        x = nn.ReLU()(self.l4(x))
        x = nn.ReLU()(self.l5(x))
        x = self.l6(x)
        return x

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        y_pred = self(x)

        loss = self.loss(y, y_pred)

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        # Logging to TensorBoard (if installed) by default
        return loss
    

    def configure_optimizers(self):
        optimizer = self.optimizer(self.parameters(), lr=self.lr)
        
        return optimizer


In [41]:
def create_dataset(dataset_X, dataset_y, lookback):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset_X)-lookback-24):
        feature = dataset_X[i:i+lookback]
        target = dataset_y[i+lookback: i+lookback+24]
        X.append(feature)
        y.append(target)
    return torch.Tensor(X), torch.Tensor(y)


In [42]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data):
        self.X, self.y = create_dataset(data, data[:,0], 24*7) 
 
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [47]:
batch_size = 32
mydataset = MyDataset(train_df)
loader = DataLoader(mydataset, batch_size=batch_size, shuffle=True)
print(next(iter(loader))[1].shape)

torch.Size([32, 24])


In [49]:
import lightning.pytorch as pl

model = SimpleModel(epochs = 10, batch_size = batch_size, lr = 0.0005)
model.configure_optimizers()
trainer = pl.Trainer(min_epochs= model.epochs, max_epochs=model.epochs, enable_progress_bar=True, enable_model_summary = False) 

trainer.fit(model= model, train_dataloaders= loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2023-09-16 13:06:39.860839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This 

Training: 0it [00:00, ?it/s]

/home/jupyter-user17/.local/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:432: PossibleUserWarning: It is recommended to use `self.log('train_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(
`Trainer.fit` stopped: `max_epochs=10` reached.
